In [202]:
#import
import pandas as pd
import google.generativeai as genai
import json
import requests
from tokenizers import Tokenizer

In [203]:
#dataframe make
data_dict = [{'title':'','text':'','abstract':'','introduction':'','literature review':'','methodology':'','results':'','conclusion':''}]
main_df = pd.DataFrame().from_dict(data_dict)
main_df.head()

,title,text,abstract,introduction,literature review,methodology,results,conclusion
0,,,,,,,,


In [204]:
url_df = pd.read_csv('dataset.csv')

In [206]:
# google
API_KEY = '' #add key here
genai.configure(api_key= API_KEY)

model = genai.GenerativeModel('gemini-pro')

generation_config = genai.GenerationConfig(
  stop_sequences = None,
  temperature=0.7,
  top_p=1,
  top_k=32,
  candidate_count=1,
  max_output_tokens=32,
)


In [306]:
data_list = []
error_title = []

In [308]:
for i in range (504,600):
    text = url_df['textdata'][i]
    if (len(text) <= 60000):
        response = model.generate_content(
        '''Extract key insights from the given research paper. Please provide four or five sentences each for the following segments: 
        {Abstract, Introduction, Literature Review, Methodology, Results, and Conclusion} of the research paper. 
        Ensure that the extracted information is concise and relevant to each segment .Give titles as 
        {Abstract, Introduction, Literature Review, Methodology, Results, and Conclusion} no other headings are 
        allowed except the provided one.Ignore any other sections or content not related to the requested segments.Remove figures and tables. 
        Make sure the extracted contents are structured as a python dictionary of the form [{"abstract":"","introduction":"","literature review":"","methodology":"","results":"","conclusion":""}] strictly and be sure to use double quotes (""). Don't deviate from this dictionary structure'''
        + "Here is the text:\n" + f'{text}'
        )
        
        try:
            dict= json.loads(response.text)
            dict['title'] = url_df['title'][i]
            dict['author'] = url_df['authors'][i]
            dict['textdata'] = url_df['textdata'][i]
            data_list.append(dict)

        except (requests.exceptions.HTTPError, ValueError) as e:
            error_title.append(url_df['title'][i])
            print(f"Error in {url_df['title'][i]}")

        print(f'File number {i},\n {response.text}')

File number 507,
 {
 "abstract": "Open information extraction (OpenIE), a task focused on the schema-free extraction of triplets from natural language text, poses various challenges, including complicated triplet structures, overlapping elements, and implicit triplets. In this paper, we introduce DualOIE, a novel generative OpenIE model which incorporates a dual task of converting triplets into sentences to address these complexities and enhance extraction performance. By leveraging the duality, DualOIE effectively recognizes the sentence structure, tackles complicated triplet patterns, and learns diverse relations between arguments, leading to improved extraction results. We establish a new benchmark, MTOIE, for implicit triplet extraction containing a significant number of implicit triplets and various predicate types. Extensive experiments on two benchmarks and our MTOIE demonstrate the superiority of DualOIE over the state-of-the-art baselines and ChatGPT. The conducted online A/B 

In [309]:
data_list

[{'abstract': 'Controllable generation of 3D human motions becomes an important topic as the world embraces digital transforma-tion. Existing works, though making promising progress with the advent of diffusion models, heavily rely on meticulously captioned and annotated (e.g., text) high-quality motion corpus, a resource-intensive endeavor in the real world. This motivates our proposed MotionMix, a simple yet effective weakly-supervised diffusion model that leverages both noisy annotated motions and unannotated motions. Specifically, we separate the denoising objectives of a diffusion model into two stages: obtaining conditional rough motion approximations in the initial T ≠ T ≤ ∂ steps by learning the noisy annotated motions, followed by the unconditional refinement of these preliminary motions during the last T ≤ steps using unannotated motions. Notably, though learning from two sources of imperfect data, our model does not compromise motion generation quality compared to fully supe

In [310]:
#number of datas extracted
len(data_list)

52

In [311]:
#makes df from  the data received from model
df = pd.DataFrame().from_dict(data_list)

In [312]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   abstract           51 non-null     object
 1   introduction       51 non-null     object
 2   literature review  21 non-null     object
 3   methodology        50 non-null     object
 4   results            50 non-null     object
 5   conclusion         51 non-null     object
 6   title              52 non-null     object
 7   author             52 non-null     object
 8   textdata           52 non-null     object
 9   literature_review  30 non-null     object
 10  Abstract           1 non-null      object
 11  Introduction       1 non-null      object
 12  Literature_Review  1 non-null      object
 13  Methodology        1 non-null      object
 14  Results            1 non-null      object
 15  Conclusion         1 non-null      object
dtypes: object(16)
memory usage: 6.6+ KB


In [313]:
#all columns into type str
for column in df.columns:
    df[column] = df[column].astype(str)

In [224]:
#duplicate columns auda ko lagi
# name_literature_review = ['literature_review','literature_reviews']
# name_methodology = ['method','methods']
# name_result = ['results']
# name_conclusion = ['conclusion']
# for name in name_literature_review:
#     df['literature review'] = df['literature review'] + df[name]
# for name in name_methodology:
#     df['methodology'] = df['methodology'] + df[name]
# for name in name_result:
#     df['result'] = df['result'] + df[name]
# for name in name_conclusion:
#     df['conclusion'] = df['conclusion'] + df[name]

In [314]:
df['literature review'] = df['literature review'] + df['literature_review']+ df['Literature_Review']  
# +df['related_work'] +df['background']
df['methodology'] = df['methodology'] + df['Methodology']
df['results'] = df['results'] + df['Results']
df['conclusion'] = df['conclusion'] + df['Conclusion']
df['abstract'] = df['abstract'] + df['Abstract']
df['introduction'] = df['introduction'] +df['Introduction']

In [315]:
df = df.drop('literature_review',axis= 1)
df = df.drop('Literature_Review',axis= 1)
df = df.drop('Methodology',axis= 1)
df = df.drop('Results',axis= 1)
df = df.drop('Conclusion',axis= 1)
df = df.drop('Abstract',axis= 1)
df = df.drop('Introduction',axis= 1)

In [316]:
#drop duplicate values on basis of titles
df = df.drop_duplicates(subset=['title'])

In [317]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   abstract           52 non-null     object
 1   introduction       52 non-null     object
 2   literature review  52 non-null     object
 3   methodology        52 non-null     object
 4   results            52 non-null     object
 5   conclusion         52 non-null     object
 6   title              52 non-null     object
 7   author             52 non-null     object
 8   textdata           52 non-null     object
dtypes: object(9)
memory usage: 3.8+ KB


In [318]:
#save the file as csv
name_file = 'df(500-600).csv'
df.to_csv(name_file, index=False)

In [319]:
error_title

['Sharing Energy in Wide Area: A Two-Layer Energy Sharing Scheme for Massive Prosumers',
 'Learning from Aggregate responses: Instance Level versus Bag Level Loss Functions',
 'PhotoBot: Reference-Guided Interactive Photography via Natural Language',
 'StreamVoice: Streamable Context-Aware Language Modeling for Real-time Zero-Shot Voice Conversion',
 'Bounding Consideration Probabilities in Consider-Then-Choose Ranking Models',
 'Design Principles & Issues for Gaze and Pinch Interaction',
 'Subjective Causality',
 'Responsible AI Governance: A Systematic Literature Review']